# Build and Zip PNG Files - Hackathon

In this notebook, we'll take the `basic` data set, use `ibmseti` Python package to convert each data file into a spectrogram, then save as `.png` files.


Then, we'll split the data set into a training set and a test set and create a handful of zip files for each class. This will dovetail into the next tutorial where we will train a custom Watson Visual Recognition classifier (we will use the zip files of pngs) and measure it's performance with the test s

## Spark Enterprise Cluster

This notebook is currently written to run on the Spark Enterprise Cluster. That is, the variables point to the data locations on the Enterprise Cluster. 

#### PowerAI

However, if you wish to run this on the PowerAI systems at the Hackathon, read cell 3 below. You only need to uncomment a few lines so that variables point to the data locations on the PowerAI system.

In [ ]:
### SET YOUR TEAM NAME HERE! Use this folder to save intermediate results
teamname = 'team_wombat'

mydatafolder = os.path.join( os.environ['PWD'], teamname )  
if os.path.exists(mydatafolder) is False:
    os.makedirs(mydatafolder)
    
outputpng_folder = mydatafolder + '/png'
if os.path.exists(outputpng_folder) is False:
    os.makedirs(outputpng_folder)

In [ ]:
from __future__ import division

import cStringIO
import glob
import json
import requests
import ibmseti
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# Will will use the Index files to retrieve our data
# Each line in training set index file contains UUID, SIGNAL_CLASSIFICATION 

primarySmallIndex = os.path.join( os.environ['PWD'], 'data/seti/simsignals_files/public_list_primary_v2_small_1june_2017.csv' )
primaryMediumIndex = os.path.join( os.environ['PWD'], 'data/seti/simsignals_files/public_list_primary_v2_medium_1june_2017.csv')
basicIndex = os.path.join( os.environ['PWD'], 'data/seti/simsignals_files/public_list_basic_v2_26may_2017.csv' )
testSetIndex = os.path.join( os.environ['PWD'], 'data/seti/simsignals_files/public_list_primary_testset_mini_1june_2017.csv' )

#define some variables for the different local directories where the data are stored
# DO NOT WRITE TO THE 'data' DIRECTORY. MAKE SURE YOU ALWAYS USE YOUR TEAM DIRECTORY YOU CREATED ABOVE TO STORE ANY INTERMEDIATE RESULTS

primarySetiDataDir = os.path.join( os.environ['PWD'],'data/seti/simsignals_v2')
basicSetiDataDir = os.path.join( os.environ['PWD'],'data/seti/simsignals_basic_v2')
testSetiDataDir = os.path.join( os.environ['PWD'],'data/seti/simsignals_test_mini')

##
## IF YOU WANT TO RUN THIS ON THE POWER AI MACHINES, UNCOMMENT THE LINES BELOW
##

## REMEMBER, on Nimbix, your local file space is destroyed when your cloud machine is shut down. So be sure to commit/save your work!
    
# Index Files are in different location 
# primarySmallIndex = '/data/seti/simsignals_files/public_list_primary_v2_small_1june_2017.csv' 
# primaryMediumIndex = '/data/seti/simsignals_files/public_list_primary_v2_medium_1june_2017.csv'
# basicIndex = '/data/seti/simsignals_files/public_list_basic_v2_26may_2017.csv' 
# testSetIndex =  '/data/seti/simsignals_files/public_list_primary_testset_mini_1june_2017.csv'

# primarySetiDataDir = '/data/seti/simsignals_v2'  #THIS ONLY CONTAINS THE SMALL AND MEDIUM DATA FILES!  
# basicSetiDataDir = '/data/seti/simsignals_basic_v2'
# testSetiDataDir = '/data/seti/simsignals_test_mini'

In [ ]:

# Choose your data set!

workingDataDir = primarySetiDataDir
workingIndexFile = primarySmallIndex

In [ ]:
#Use `ibmseti`, or other methods, to draw the spectrograms

def draw_spectrogram(data):
    
    aca = ibmseti.compamp.SimCompamp(data)
    spec = aca.get_spectrogram()
    
    
    # Instead of using SimCompAmp.get_spectrogram method
    # perform your own signal processing here before you create the spectrogram
    #
    # SimCompAmp.get_spectrogram is relatively simple. Here's the code to reproduce it:
    #
    # header, raw_data = r.content.split('\n',1)
    # complex_data = np.frombuffer(raw_data, dtype='i1').astype(np.float32).view(np.complex64)
    # complex_data = complex_data - complex_data.mean()  # have to subtract off any DC offset
    # shape = (32, 6144)
    # spec = np.abs( np.fft.fftshift( np.fft.fft( complex_data.reshape(*shape) ), 1) )**2
    # 
    # But instead of the line above, can you maniputlate `complex_data` with signal processing
    # techniques in the time-domain (windowing?, de-chirp?), or manipulate the output of the 
    # np.fft.fft process in a way to improve the signal to noise (Welch periodogram, subtract noise model)? 
    # 
    # example: Apply Hanning Window
    # complex_data = complex_data.reshape(*shape)
    # complex_data = complex_data * np.hanning(complex_data.shape[1])
    # spec = np.abs( np.fft.fftshift( np.fft.fft( complex_data ), 1) )**2

    
    ## Noise Subtraction
    #
    #  If you are building an average noise spectrogram model for subtraction, you should do that here.
    #
    #  See the Example to build an average noise spectrogram: 
    #
    #  Important point: If you do signal processing above to the raw data, you should apply the exact same signal processing
    #     when you calculate your average noise spectrogram
    #
    #    import pickle
    #    ave_noise_spec = pickle.load(os.path.join(mydatafolder, 'ave_noise_spec.pickle'))
    #    spec = spec - ave_noise_spec
    #
    #
     

    fig, ax = plt.subplots(figsize=(10, 5))   

    # do different color mappings affect Watson's classification accuracy?
    
    # ax.imshow(np.log(spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0], cmap='hot')
    # ax.imshow(np.log(spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0], cmap='gray')
    # ax.imshow(np.log(spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0], cmap='Greys')
    
    ax.imshow(np.log(spec), aspect = 0.5*float(spec.shape[1]) / spec.shape[0], cmap='gray')
    
    ##
    ## For other ways to create Images, see: 
    ## tutorials/Step_5c_Convert_TS_to_unit8Dataset_DSX.ipynb
    ##
    
    return fig, aca.header()


def convert_to_spectrogram_and_save(row):
    
    try:
        uuid, classification = row.split(',')
    except:
        uuid = row #this handles the test data since it doesn't have "SIGNAL_CLASSIFICATION" in index file
        classification = 'unknown: test data'
        
        
    #create path to local data file
    filename = uuid + '.dat'
    filepath = os.path.join(workingDataDir, filename)
    
    #retrieve that data file
    rawdata = open(filepath).read()
    
    
    fig, header = draw_spectrogram(rawdata)
    
    png_file_name = filename + '.png'
    fig.savefig( os.path.join(outputpng_folder, png_file_name) )
    plt.close(fig)
    
    return (filename, header, png_file_name)

In [ ]:
rdd = sc.textFile(workingIndexFile, 30).filter(lambda x: x.startswith('UUID') is False) #the filter removes the header


In [ ]:
rdd.count()

In [ ]:
results = rdd.map(convert_to_spectrogram_and_save).collect()

In [ ]:
results[0]

# Create Training / Test sets

Using the `basic` list, we'll create training and test sets for each signal class. Then we'll archive the `.png` files into a handful of `.zip` files (We need the .zip files to be smaller than 25 MB each because there is a limitation with the size of batches of data that are uploaded to Watson Visual Recognition when training a classifier (200 MB total).)

In [ ]:
# Grab the Basic file list in order to 
# Organize the Data into classes

indexfile_rows = open(workingIndexFile).readlines()
                                                    
uuids_classes_as_list = indexfile_rows  [1:]#slice off the first line (header)

def row_to_json(row):
    uuid,sigclass = row.strip('\n').split(',')  #strip \n and split uuid, class
    return {'uuid':uuid, 'signal_classification':sigclass}

uuids_classes_as_list = map(lambda row: row_to_json(row), uuids_classes_as_list)
print "found {} files".format(len(uuids_classes_as_list))

uuids_group_by_class = {}
for item in uuids_classes_as_list:
    uuids_group_by_class.setdefault(item['signal_classification'], []).append(item)

In [ ]:

#At first, use just 20 percent and 10 percent. This will be useful 
#as you prototype. You'll you use these to train Watson in the next notebook
#So, if we only do the first 20 percent and 10 percent, we can move through
#the tutorial quickly at first. Then you can come back here and increase these
#percentages. 
training_percentage = 0.20
test_percentage = 0.10

assert training_percentage + test_percentage <= 1.0

training_set_group_by_class = {}
test_set_group_by_class = {}
for k, v in uuids_group_by_class.iteritems():
    
    total = len(v)
    training_size = int(total * training_percentage)
    test_size = int(total * test_percentage)
    
    training_set = v[:training_size]
    test_set = v[-1*test_size:]
    
    training_set_group_by_class[k] = training_set
    test_set_group_by_class[k] = test_set
    
    print '{}: training set size: {}'.format(k, len(training_set))
    print '{}: test set size: {}'.format(k, len(test_set))

In [ ]:
training_set_group_by_class['noise'][0]

In [ ]:
#fnames = [ os.path.join(outputpng_folder, vv['uuid'] + '.dat.png') for vv in v]

In [ ]:
#make sure this looks right.
# print fnames[0]
# os.stat(fnames[0]).st_size

In [ ]:
zipfilefolder = mydatafolder + '/zipfiles'
if os.path.exists(zipfilefolder) is False:
    os.makedirs(zipfilefolder)

In [ ]:
max_zip_file_size_in_mb = 25

In [ ]:
#Create the Zip files containing the training PNG files
#Note that this limits output files to be less than <max_zip_file_size_in_mb> MB because WatsonVR has a limit on the 
#size of input files that can be sent in single HTTP calls to train a custom classifier

for k, v, in training_set_group_by_class.iteritems():
    
    fnames = [ os.path.join(outputpng_folder, vv['uuid'] + '.dat.png') for vv in v]  #yes, files are <uuid>.dat.png :/
    
    count = 1
    for fn in fnames:
        
        archive_name = '{}/classification_{}_{}.zip'.format(zipfilefolder, count, k)
        
        if os.path.exists(archive_name):
            zz = zipfile.ZipFile(archive_name, mode='a')
        else:
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')
           
        
        zz.write(fn, fn.split('/')[-1])
        zz.close()
        
        #if archive_name folder exceeds <max_zip_file_size_in_mb> MB, increase count to create a new one
        if os.path.getsize(archive_name) > max_zip_file_size_in_mb * 1024 ** 2:
            count += 1
            

In [ ]:
#Create the Zip files containing the test PNG files
#Note that this limits output files to be less than <max_zip_file_size_in_mb> MB because WatsonVR has a limit on the 
#size of input files that can be sent in single HTTP calls to train a custom classifier

for k, v, in test_set_group_by_class.iteritems():
    
    fnames = [outputpng_folder + '/' + vv['uuid'] + '.dat.png' for vv in v]  #yes, files are <uuid>.dat.png :/
    
    count = 1
    for fn in fnames:
        
        archive_name = '{}/testset_{}_{}.zip'.format(zipfilefolder, count, k)
        
        if os.path.exists(archive_name):
            zz = zipfile.ZipFile(archive_name, mode='a')
        else:
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')
           
        zz.write(fn)
        zz.close()
        
        #if archive_name folder exceeds <max_zip_file_size_in_mb> MB, increase count to create a new one
        if os.path.getsize(archive_name) > max_zip_file_size_in_mb * 1024 ** 2:
            count += 1
            

In [ ]:
# Create the Zip files containing the test PNG files using the following naming convention:
# testset_<NUMBER>_<CLASS>.zip (step 4 will break if a different naming convention is used)
# Refer to https://www.ibm.com/watson/developercloud/visual-recognition/api/v3/#classify_an_image for ZIP size and content limitations:
# "The max number of images in a .zip file is limited to 20, and limited to 5 MB."

for k, v, in test_set_group_by_class.iteritems():
    
    fnames = [outputpng_folder + '/' + vv['uuid'] + '.dat.png' for vv in v]  #yes, files are <uuid>.dat.png :/
    
    # archive counter
    count = 1
    # number of image files in archive counter
    image_count = 0
    for fn in fnames:
        
        archive_name = '{}/testset_{}_{}.zip'.format(zipfilefolder, count, k)
        
        if os.path.exists(archive_name):
            if os.path.getsize(archive_name) + os.path.getsize(fn) >= 4.9 * 1024 ** 2:
                # current ZIP archive size + size of this file > max size (or at least close to); create new archive
                count += 1
                image_count = 0
                archive_name = '{}/testset_{}_{}.zip'.format(zipfilefolder, count, k)
                print 'creating new archive', archive_name
                zz = zipfile.ZipFile(archive_name, mode='w')
            else:
                zz = zipfile.ZipFile(archive_name, mode='a')
        else:
            print 'creating new archive', archive_name
            zz = zipfile.ZipFile(archive_name, mode='w')
           
        zz.write(fn)
        zz.close()
        
        image_count += 1
        # the number of files > max number of files supported by API; create new archive
        if image_count > 19:
            count +=1
            image_count = 0

In [ ]:
!ls -alh team_wombat/zipfiles/

# Exporting your Zip files

You do NOT need to do this if you're going on to the next notebook where you use Watson to classify your images from this Spark cluster. That notebook will read the data from here.

However, if you want to move these data elsewhere, teasiest and fastest way to push your data out is to send it to your IBM Object Storage

1. Log in to https://bluemix.net
2. Scroll down and find your Object Storage instance. 
  * If you do not have one, find the "Catalog" link and look for the Object Storage service to create a new instance (5 GB of free space)
3. Select the `Service Credentials` tab and `View Credentials`
4. Copy these into your notebook below.


In [ ]:
import swiftclient.client as swiftclient

credentials = {
  'auth_uri':'',
  'global_account_auth_uri':'',
  'username':'xx',
  'password':"xx",
  'auth_url':'https://identity.open.softlayer.com',
  'project':'xx',
  'projectId':'xx',
  'region':'dallas',
  'userId':'xx',
  'domain_id':'xx',
  'domain_name':'xx',
  'tenantId':'xx'
}

In [ ]:
conn_seti_data = swiftclient.Connection(
    key=creds_seti_public['password'],
    authurl=creds_seti_public['auth_url']+"/v3",
    auth_version='3',
    os_options={
        "project_id": creds_seti_public['projectId'],
        "user_id": creds_seti_public['userId'],
        "region_name": creds_seti_public['region']})

In [ ]:
myObjectStorageContainer = 'example_pngs'  
conn_seti_data.post_container(myObjectStorageContainer)  #creates a new container in your object storage

someFile = os.path.join(zipfilefolder, 'classification_1_narrowband.zip')

etag = conn_seti_data.put_object(myObjectStorageContainer, someFile, open(someFile,'rb').read())